In [27]:
import pandas as pd
import os
import datetime
from itertools import repeat
# import airflow
import shutil
import sqlite3
from DBConnector import Save_User,readControlDataLake,Update,connect_db

In [29]:
#Variables

Path = 'files'
global DestinationPath
DestinationPath = 'DataLake'
# This function will create an ID to identify tables
def UniqueID (TableName):
    mybytes = TableName.encode('utf-8')
    ID = int.from_bytes(mybytes, 'little')
    return ID

def Update (TableID,Table_Name,UploadDate,ModificationDate,Origin,Destiny): 
    TableIDstr = str(TableID)
    db = connect_db()    
    print(TableID,Table_Name,UploadDate,ModificationDate,Origin,Destiny)
    cursor = db.cursor()
    cursor.execute("INSERT INTO ControlDataLake(TableID,TableName,UploadDate,ModificationDate,Origin,Destiny) VALUES(?,?,?,?,?,?)",(TableIDstr,Table_Name,UploadDate,ModificationDate,Origin,Destiny))
    db.commit()

# Create Governed Table (Metadata Only)

In [5]:

def ExtractMetadata (): #Function to extract the metadata in order to include in the governed table
    metadata = dict()
    arr = os.listdir(Path)
    for csvFiles in arr:
        FilePath = Path + '/' + csvFiles
        # print (FilePath)
        # print(os.stat(FilePath))
        metadata.update({csvFiles:os.stat(FilePath)})
    return metadata

def ExtractFilenames (): #The function will extract all the names from the path origin 
    arr = os.listdir(Path)
    FileNames = []
    for csvFiles in arr:
        FilePath = csvFiles #Path + '/' + csvFiles
        FileNames.append(FilePath)
    return FileNames

def GovernedTable (): #This function will create a table to index and filter fields and information about data files
    FileNames = ExtractFilenames()
    metadata = ExtractMetadata()
    DataGovern = pd.DataFrame ()
    TableFields = pd.DataFrame ()
    
    
    for names in FileNames:  #It will explore diferents files which are load
        listacolumnas = dict()
        Extension = names[len(names)-3 : len(names)]
        
        if Extension == 'csv': #file is a csv
            df= pd.read_csv(Path + '/' + names)
            
            columns = list(df.columns)
            types   = list(df.dtypes)
            IDTables  =list (repeat(UniqueID(names),len(types)))
            TableFieldsnew = pd.DataFrame ({'TableId':IDTables,'Column Names':columns,'Data Type':types })
            TableFields = pd.concat([TableFields, TableFieldsnew])
        else:
                 IDTables =  UniqueID()

 
        if Extension == 'csv' or Extension == 'xlsx':   
            StructureData = 'Yes'
        else: 
            StructureData = 'No'
        # Data Governed table which inform about the metadata
        DataGovernnew =  pd.DataFrame ({'TableId':IDTables[1],
                                     'Table Name': names,
                                     'Comment':'',  
                                          'Creation Date': datetime.datetime.fromtimestamp(metadata[names][9]),
                                          'Modification Date': datetime.datetime.fromtimestamp(metadata[names][8]),
                                          'File Size': metadata[names][6], 
                                          'File Type': Extension,
                                          'Structured Data':'Yes',
                                          'Load': 'No',
                                          'DateLoad': 'Never'}, index=[0])
                                           
        
        DataGovern  = pd.concat([DataGovern, DataGovernnew])
        
    #Reset the index    
    DataGovern.reset_index(drop= True, inplace = True) 
    TableFields.reset_index(drop= True, inplace = True) 
    display(TableFields) 
    display (DataGovern)

            
        

# Moving Files to DataLake

In [26]:

    
def copycsvfile (TableName):
    try:
        shutil.copy(Path + '/' + TableName , DestinationPath + '/' + TableName)
        print("File copied successfully.")
    except shutil.SameFileError:
         print("Source and destination represents the same file.")
    # If there is any permission issue
    except PermissionError:
        print("Permission denied.") 
    # For other errors
    except:
        print("Error occurred while copying file.")
    Update (UniqueID(TableName),str(TableName),str(datetime.datetime.now()),str(datetime.datetime.now()),Path + '/' + TableName,DestinationPath + '/' + TableName )
    return print("Save it in the data lake")



File copied successfully.
3669323166161444168241861682847098757601682877257321371570083651948753533521390287071642917106938263462485 Uber Movement - Travel Times Methodology.pdf 2022-10-26 13:06:31.236266 2022-10-26 13:06:31.236266 files/Uber Movement - Travel Times Methodology.pdf DataLake/Uber Movement - Travel Times Methodology.pdf
Save it in the data lake


# Filter Select Records  and insert them into governed table

In [ ]:
#Select only the columns we want to Load
#Data should be available by time 

def Filter():

    return 0

# Create transaction create storage

In [ ]:
def create_directory():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)
        
def delete_directory():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)
        
def rename_directory():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)
        
def upload_file_to_directory():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)
def download_file_from_directory():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)
def list_directory_contents():
    try:
        print("Creating directory")
    except Exception as e:
        print(e)

# Write data into a database

Storage in HDFS 
Storage in MongoDb

# Read from governed table 

# Create airflow, pipeline for manage it

In [ ]:
#Main 
